In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import json
import time
import logging
from urllib.parse import urljoin
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [8]:
class RSODetailScraper:
    def __init__(self, input_file='rso_data_with_categories.json', 
                 checkpoint_file='rso_scraping_checkpoint.json',
                 checkpoint_frequency=10):  # Save every 10 RSOs
        self.input_file = input_file
        self.checkpoint_file = checkpoint_file
        self.checkpoint_frequency = checkpoint_frequency
        self.driver = None
        
    def save_checkpoint(self, rsos, last_processed_index):
        """Save current progress to checkpoint file"""
        checkpoint_data = {
            'last_processed_index': last_processed_index,
            'rsos': rsos
        }
        with open(self.checkpoint_file, 'w') as f:
            json.dump(checkpoint_data, f, indent=2)
        logger.info(f"Checkpoint saved at index {last_processed_index}")
        
    def load_checkpoint(self):
        """Load progress from checkpoint file if it exists"""
        try:
            with open(self.checkpoint_file, 'r') as f:
                checkpoint_data = json.load(f)
            logger.info(f"Resuming from checkpoint at index {checkpoint_data['last_processed_index']}")
            return checkpoint_data['last_processed_index'], checkpoint_data['rsos']
        except FileNotFoundError:
            logger.info("No checkpoint found, starting from beginning")
            return -1, None
        
    def setup_driver(self):
        """Initialize Selenium WebDriver"""
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--window-size=1920,1080')
        self.driver = webdriver.Chrome(options=options)
        
    def clean_text(self, text):
        """Clean text by removing extra whitespace and asterisks"""
        if not text:
            return ''
        return text.strip('* ').strip()
    
    def scrape_detail_page(self, url):
        """Scrape a single RSO detail page"""
        try:
            self.driver.get(url)
            time.sleep(1)  # Allow page to load
            
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            details = {
                'full_description': '',
                'contact': {},
                'additional_info': {}
            }
            
            # Get full description from the correct div class and all paragraph tags within
            description_div = soup.find('div', class_='bodyText-large userSupplied')
            if description_div:
                paragraphs = description_div.find_all('p')
                details['full_description'] = ' '.join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
            
            # Get contact information - updated for new structure
            contact_email_div = soup.find('span', class_='sr-only', string='Contact Email')
            if contact_email_div and contact_email_div.parent:
                email_text = contact_email_div.parent.get_text()
                # Extract the email from the text (everything after "E: ")
                if 'E:' in email_text:
                    email = email_text.split('E:')[1].strip().strip('"')
                    details['contact']['email'] = email
                
            # Get address if available
            address_div = soup.find('div', string='Address')
            if address_div and address_div.find_next('div'):
                details['contact']['address'] = self.clean_text(address_div.find_next('div').get_text())
            
            
            # Get social media links and website
            social_media = {}
            
            # Get website from aria-label
            website_link = soup.find('a', attrs={'aria-label': lambda x: x and 'Visit our site' in x})
            if website_link and website_link.get('href'):
                social_media['website'] = website_link['href']
            
            # Get social media links
            social_links = soup.find_all('a', href=True)
            for link in social_links:
                href = link['href']
                if 'facebook.com' in href:
                    social_media['facebook'] = href
                elif 'instagram.com' in href:
                    social_media['instagram'] = href
            
            if social_media:
                details['social_media'] = social_media


           # Get additional information
            additional_info_h2 = soup.find('h2', string=lambda x: x and 'Additional Information' in x)
            if additional_info_h2:
                # Get to the container div
                container = additional_info_h2.parent.parent.find_next_sibling('div')
                if container:
                    # Find all field divs by their specific style
                    field_divs = container.find_all('div', style=lambda x: x and 'padding-bottom: 8px; margin-left: 15px;' in x)
                    
                    for field_div in field_divs:
                        # Get the label from the strong tag
                        label_div = field_div.find('div', style='font-weight: bold;')
                        if label_div and label_div.strong:
                            label = label_div.strong.text.strip()
                            
                            # Get the value by getting the second div (skipping the label div)
                            divs = field_div.find_all('div', recursive=False)
                            if len(divs) >= 2:  # Make sure we have both label and value divs
                                value_div = divs[1].find('div')  # Get the inner div of the second div
                                if value_div:
                                    value = value_div.text.strip()
                                    details['additional_info'][label] = value
                    
                    logger.info(f"Extracted additional info: {details['additional_info']}")
            return details
            
        except Exception as e:
            logger.error(f"Error scraping {url}: {str(e)}")
            return None

            
    def scrape_all_rsos(self):
        """Scrape details for all RSOs with checkpointing"""
        try:
            # Try to load from checkpoint first
            last_processed_index, checkpoint_rsos = self.load_checkpoint()
            
            # If no checkpoint, load from input file
            if checkpoint_rsos is None:
                with open(self.input_file, 'r') as f:
                    rsos = json.load(f)
                start_index = 0
            else:
                rsos = checkpoint_rsos
                start_index = last_processed_index + 1
            
            logger.info(f"Processing {len(rsos)} RSOs starting from index {start_index}")
            self.setup_driver()
            
            # Process each RSO
            for i in range(start_index, len(rsos)):
                rso = rsos[i]
                url = rso.get('full_url')
                if not url:
                    continue
                
                logger.info(f"Processing RSO {i+1}/{len(rsos)}: {rso.get('name', 'Unknown')}")
                details = self.scrape_detail_page(url)
                
                if details:
                    rso.update(details)
                
                # Save checkpoint periodically
                if (i + 1) % self.checkpoint_frequency == 0:
                    self.save_checkpoint(rsos, i)
                
                # Add a small delay between requests
                time.sleep(1)
            
            # Save final results
            output_file = 'rso_data_detailed.json'
            with open(output_file, 'w') as f:
                json.dump(rsos, f, indent=2)
            
            # Clean up checkpoint file after successful completion
            if os.path.exists(self.checkpoint_file):
                os.remove(self.checkpoint_file)
                
            logger.info(f"Successfully saved detailed RSO data to {output_file}")
            
        except Exception as e:
            logger.error(f"Error in scraping process: {str(e)}")
            # Save checkpoint on error
            if 'rsos' in locals() and 'i' in locals():
                self.save_checkpoint(rsos, i)
                logger.info("Progress saved to checkpoint file after error")
        finally:
            if self.driver:
                self.driver.quit()

In [7]:
class RSODetailScraper:
    def __init__(self, input_file='rso_data_with_categories.json'):
        self.input_file = input_file
        self.driver = None
        
    
            
    
    def scrape_all_rsos(self):
        """Scrape details for all RSOs"""
        try:
            # Load existing RSO data
            with open(self.input_file, 'r') as f:
                rsos = json.load(f)
            
            logger.info(f"Loaded {len(rsos)} RSOs from {self.input_file}")
            self.setup_driver()
            
            # Process each RSO
            for i, rso in enumerate(rsos):
                url = rso.get('full_url')
                if not url:
                    continue
                
                logger.info(f"Processing RSO {i+1}/{len(rsos)}: {rso.get('name', 'Unknown')}")
                details = self.scrape_detail_page(url)
                
                if details:
                    rso.update(details)
                
                # Add a small delay between requests
                time.sleep(1)
            
            # Save updated data to new file
            output_file = 'rso_data_detailed.json'
            with open(output_file, 'w') as f:
                json.dump(rsos, f, indent=2)
            
            logger.info(f"Successfully saved detailed RSO data to {output_file}")
            
        except Exception as e:
            logger.error(f"Error in scraping process: {str(e)}")
        finally:
            if self.driver:
                self.driver.quit()
    


    # test method
    def test_detail_scraping(self, num_test_pages=3):
        """Test the detail scraping on a few RSO pages and print results"""
        try:
            # Load existing RSO data
            with open(self.input_file, 'r') as f:
                rsos = json.load(f)
            
            logger.info(f"Running test on {num_test_pages} RSO pages...")
            self.setup_driver()
            
            for i, rso in enumerate(rsos[:num_test_pages]):
                url = rso.get('full_url')
                if not url:
                    continue
                
                logger.info(f"\nTesting RSO {i+1}: {rso.get('name', 'Unknown')}")
                logger.info(f"URL: {url}")
                
                details = self.scrape_detail_page(url)
                
                # Print detailed results
                if details:
                    logger.info("\nScraped Details:")
                    logger.info(f"Description: {details['full_description'][:200]}...")
                    logger.info(f"Contact Info: {details['contact']}")
                    logger.info(f"Social Media: {details.get('social_media', {})}")
                    logger.info(f"Additional Info: {details['additional_info']}")
                else:
                    logger.error("Failed to scrape details")
                
                time.sleep(1)
                
        except Exception as e:
            logger.error(f"Error in test process: {str(e)}")
        finally:
            if self.driver:
                self.driver.quit()



In [ ]:
scraper = RSODetailScraper()
scraper.scrape_all_rsos()

In [ ]:
scraper = RSODetailScraper()
# Run test first
scraper.test_detail_scraping()